# Load Data

In [3]:
import pandas as pd

file_path = '../data/raw/rent_history.xlsx' # Read file
df = pd.read_excel(file_path)

# Read and Process Data 

In [4]:
def process_rent_data(file_path):

    # Define the mapping of sheet names to the corresponding number of bedrooms
    bed_map = {
        '1 bedroom flat': 1,
        '2 bedroom flat': 2,
        '3 bedroom flat': 3,
        '2 bedroom house': 2,
        '3 bedroom house': 3,
        '4 bedroom house': 4
    }
    
    # Read the Excel file and get all sheet names
    xls = pd.ExcelFile(file_path)

    # To store the processed data from each sheet
    all_sheets_data = [] 
    
    # Loop through all sheet names in the Excel file
    for sheet_name in xls.sheet_names:

        # Check if the sheet name is relevant
        if sheet_name in bed_map: 

            # Read the data from the sheet and skip the first row as it contains metadata
            df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=1)
            
            # Initialize new column names
            new_columns = []
            columns = df.columns

            # Replace 'Unnamed' columns with previous column names
            for idx, col in enumerate(columns):

                # Keep the original column names of the first two columns
                if idx < 2: 
                    new_columns.append(col)
                else:

                    # Use the previous column name instead if the column name contains 'Unnamed'
                    if 'Unnamed' in str(col):
                        new_columns.append(new_columns[-1])
                    else:
                        new_columns.append(col)

            # Assign the updated column names back to the dataframe
            df.columns = new_columns

            # Add a 'zone' column by detecting the zone name (e.g., 'Inner Melbourne')
            df['Zone'] = None
            zone_name = None
            for idx, row in df.iloc[1:].iterrows():
                if pd.notnull(row[sheet_name]):
                    zone_name = row[sheet_name]
                    if pd.notna(row['Unnamed: 1']):
                        df.at[idx, 'Zone'] = zone_name

            # Forward-fill zone column to ensure every row has the correct zone
            df['Zone'] = df['Zone'].fillna(method='ffill')

            # Drop the first column as it's unnecessary
            df = df.drop(df.columns[0], axis=1)

            # Rename 'Unnamed: 1' to 'Location'
            if 'Unnamed: 1' in df.columns:
                df = df.rename(columns={'Unnamed: 1': 'Location'})

            # Extract first row for Count/Median info and remove the first row from the dataframe
            type_row = df.iloc[0, 1:-1]  # Exclude 'Location' and 'zone'
            df = df.drop(0).reset_index(drop=True)

            # Set new multi-level columns by combining YearMonth and Count/Median type
            year_month_cols = df.columns[1:-1]  # Exclude 'Location' and 'zone'
            if len(year_month_cols) != len(type_row):
                raise ValueError(f"Column length mismatch in {sheet_name}.")
            
            df.columns = ['Location'] + list(year_month_cols) + ['Zone']
            type_row_full = [''] + type_row.tolist() + ['']
            df.columns = pd.MultiIndex.from_arrays([df.columns, type_row_full], names=['YearMonth', 'Type'])

            # Flatten the multi-level columns before melting
            df.columns = [f'{col[0]}' if col[1] == '' else f'{col[0]}_{col[1]}' for col in df.columns]
    
            # Melt dataframe to convert wide format to long format
            df_melted = df.melt(id_vars=['Location', 'Zone'], var_name='YearMonth_Type', value_name='Value')

            # Extract Year and Month information
            df_melted['Year'] = df_melted['YearMonth_Type'].str.extract(r'(\d{4})')
            df_melted['Month'] = df_melted['YearMonth_Type'].str.extract(r'([A-Za-z]+)')
            df_melted['Type'] = df_melted['YearMonth_Type'].str.extract(r'_(Count|Median)')

            # Drop the intermediary YearMonth_Type column
            df_melted = df_melted.drop(columns=['YearMonth_Type'])

            # Create a pivot table to separate Count and Median into different columns
            df_pivot = df_melted.pivot_table(
                index=['Location', 'Year', 'Month', 'Zone'],
                columns='Type',
                values='Value',
                aggfunc='first'
            ).reset_index()

            # Add the 'Bed' column to the DataFrame based on the sheet name
            df_pivot['Bed'] = bed_map.get(sheet_name, None)

            # Append the processed DataFrame to the list of all sheets' data
            all_sheets_data.append(df_pivot)
    
    # Concatenate all the processed DataFrames from each sheet into a single DataFrame
    combined_data = pd.concat(all_sheets_data, ignore_index=True)
    
    return combined_data

# Final processed data
processed_data = process_rent_data(file_path)

processed_data.head(10)


/tmp/ipykernel_6703/276790402.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Zone'] = df['Zone'].fillna(method='ffill')
/tmp/ipykernel_6703/276790402.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Zone'] = df['Zone'].fillna(method='ffill')
/tmp/ipykernel_6703/276790402.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Zone'] = df['Zone'].fillna(method='ffill')
/tmp/ipykernel_6703/276790402.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Zone'] = df['Zone'].fillna(method='ffill')
/tmp/ipykernel_6703/276790402.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future ver

Type,Location,Year,Month,Zone,Count,Median,Bed
0,Albert Park-Middle Park-West St Kilda,2000,Dec,Inner Melbourne,369,175,1
1,Albert Park-Middle Park-West St Kilda,2000,Jun,Inner Melbourne,347,165,1
2,Albert Park-Middle Park-West St Kilda,2000,Mar,Inner Melbourne,352,165,1
3,Albert Park-Middle Park-West St Kilda,2000,Sep,Inner Melbourne,378,170,1
4,Albert Park-Middle Park-West St Kilda,2001,Dec,Inner Melbourne,423,190,1
5,Albert Park-Middle Park-West St Kilda,2001,Jun,Inner Melbourne,411,185,1
6,Albert Park-Middle Park-West St Kilda,2001,Mar,Inner Melbourne,395,180,1
7,Albert Park-Middle Park-West St Kilda,2001,Sep,Inner Melbourne,405,190,1
8,Albert Park-Middle Park-West St Kilda,2002,Jun,Inner Melbourne,413,200,1
9,Albert Park-Middle Park-West St Kilda,2002,Mar,Inner Melbourne,424,195,1
